In [1]:
!pip install -U libsvm-official
!pip install -U liblinear-official

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for libsvm-official: filename=libsvm_official-3.31.0-cp310-cp310-linux_x86_64.whl size=125901 sha256=5344c687f9308185b838cb72310006c18ebedb4a1f5a68877a5fac1f8dc25e08
  Stored in directory: /root/.cache/pip/wheels/93/04/ec/306a03478d811cf83c10665ca5e60231f47360a367e06d94d0
Successfully built libsvm-official
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.9/46.9 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for liblinear-official: filename=liblinear_official-2.46.1-cp310-cp310-linux_x86_64.whl size=146571 sha256=da422521685c6ec76e99e01b77c6eca88cee827c539236a64c02bdcf0341c30e
  Stored in directory: /root/.cache/pip/wheels/d1/c4/f5/0f9664e5a5484cfe73e265aaa8eb83d8d51342760c979cb846
Successfully built li

In [2]:
import numpy as np
#from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from liblinear.liblinearutil import *
from sklearn.datasets import load_svmlight_file
import pandas as pd

In [3]:
total_data=pd.read_csv("/content/breast-cancer_scale.txt", header=None, sep="[: ]+")

<ipython-input-3-8cc30ce507f5>:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  total_data=pd.read_csv("/content/breast-cancer_scale.txt", header=None, sep="[: ]+")


In [4]:
total_data

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,2,1,-0.860107,2,-0.111111,3,-1.000000,4,-1.000000,5,...,6,-0.777778,7,-1.000000,8,-0.555556,9,-1.000000,10,-1.000000
1,2,1,-0.859671,2,-0.111111,3,-0.333333,4,-0.333333,5,...,6,0.333333,7,1.000000,8,-0.555556,9,-0.777778,10,-1.000000
2,2,1,-0.857807,2,-0.555556,3,-1.000000,4,-1.000000,5,...,6,-0.777778,7,-0.777778,8,-0.555556,9,-1.000000,10,-1.000000
3,2,1,-0.857680,2,0.111111,3,0.555556,4,0.555556,5,...,6,-0.555556,7,-0.333333,8,-0.555556,9,0.333333,10,-1.000000
4,2,1,-0.857569,2,-0.333333,3,-1.000000,4,-1.000000,5,...,6,-0.777778,7,-1.000000,8,-0.555556,9,-1.000000,10,-1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
678,2,1,-0.893460,2,-0.555556,3,-1.000000,4,-1.000000,5,...,6,-0.555556,7,-0.777778,8,-1.000000,9,-1.000000,10,-1.000000
679,2,1,-0.883744,2,-0.777778,3,-1.000000,4,-1.000000,5,...,6,-0.777778,7,-1.000000,8,-1.000000,9,-1.000000,10,-1.000000
680,4,1,-0.876716,2,-0.111111,3,1.000000,4,1.000000,5,...,6,0.333333,7,-0.555556,8,0.555556,9,1.000000,10,-0.777778
681,4,1,-0.875424,2,-0.333333,3,0.555556,4,0.111111,5,...,6,-0.555556,7,-0.333333,8,1.000000,9,0.111111,10,-1.000000


In [5]:
total_data = total_data.iloc[:, ::2]
total_data.columns = range(total_data.shape[1])


In [6]:
total_data

,0,1,2,3,4,5,6,7,8,9,10
0,2,-0.860107,-0.111111,-1.000000,-1.000000,-1.000000,-0.777778,-1.000000,-0.555556,-1.000000,-1.000000
1,2,-0.859671,-0.111111,-0.333333,-0.333333,-0.111111,0.333333,1.000000,-0.555556,-0.777778,-1.000000
2,2,-0.857807,-0.555556,-1.000000,-1.000000,-1.000000,-0.777778,-0.777778,-0.555556,-1.000000,-1.000000
3,2,-0.857680,0.111111,0.555556,0.555556,-1.000000,-0.555556,-0.333333,-0.555556,0.333333,-1.000000
4,2,-0.857569,-0.333333,-1.000000,-1.000000,-0.555556,-0.777778,-1.000000,-0.555556,-1.000000,-1.000000
...,...,...,...,...,...,...,...,...,...,...,...
678,2,-0.893460,-0.555556,-1.000000,-1.000000,-1.000000,-0.555556,-0.777778,-1.000000,-1.000000,-1.000000
679,2,-0.883744,-0.777778,-1.000000,-1.000000,-1.000000,-0.777778,-1.000000,-1.000000,-1.000000,-1.000000
680,4,-0.876716,-0.111111,1.000000,1.000000,-0.555556,0.333333,-0.555556,0.555556,1.000000,-0.777778
681,4,-0.875424,-0.333333,0.555556,0.111111,-0.333333,-0.555556,-0.333333,1.000000,0.111111,-1.000000


In [7]:
print(total_data.loc[0, 1])

-0.860107


In [8]:
train_indices = pd.read_csv("/content/breast-cancer-scale-train-indices.txt",header=None)
test_indices = pd.read_csv("/content/breast-cancer-scale-test-indices.txt",header=None)

In [9]:
# Select rows from data based on train_indices
train_data = total_data[total_data.index.isin(train_indices[0].values-1)]

# Select rows from data based on test_indices
test_data = total_data[total_data.index.isin(test_indices[0].values-1)]


In [10]:
train_data

,0,1,2,3,4,5,6,7,8,9,10
2,2,-0.857807,-0.555556,-1.000000,-1.000000,-1.000000,-0.777778,-0.777778,-0.555556,-1.000000,-1.000000
4,2,-0.857569,-0.333333,-1.000000,-1.000000,-0.555556,-0.777778,-1.000000,-0.555556,-1.000000,-1.000000
5,4,-0.857554,0.555556,1.000000,1.000000,0.555556,0.333333,1.000000,0.777778,0.333333,-1.000000
6,2,-0.857408,-1.000000,-1.000000,-1.000000,-1.000000,-0.777778,1.000000,-0.555556,-1.000000,-1.000000
7,2,-0.857339,-0.777778,-1.000000,-0.777778,-1.000000,-0.777778,-1.000000,-0.555556,-1.000000,-1.000000
...,...,...,...,...,...,...,...,...,...,...,...
674,2,-0.911706,-1.000000,-1.000000,-1.000000,-0.555556,-0.777778,-1.000000,-1.000000,-1.000000,-1.000000
675,4,-0.905650,-0.111111,1.000000,1.000000,-0.111111,-0.333333,-0.111111,-0.333333,-0.333333,-1.000000
676,2,-0.902821,-0.555556,-1.000000,-1.000000,-1.000000,-0.777778,-1.000000,-1.000000,-1.000000,-1.000000
677,2,-0.895473,-0.555556,-1.000000,-1.000000,-1.000000,-0.777778,-1.000000,-0.777778,-1.000000,-0.777778


In [11]:
# Split train_data into X_train and y_train
X_train = train_data.drop(columns=train_data.columns[0]).values
y_train = train_data.iloc[:, 0].values.reshape(-1, 1)

# Split test_data into X_test and y_test
X_test = test_data.drop(columns=test_data.columns[0]).values
y_test = test_data.iloc[:, 0].values.reshape(-1, 1)


In [13]:

C_list = [0.1, 1, 10, 100, 1000]
Cross_Validation_accuracy=[]
for c in C_list:
    print(f"Training C={c}")
    params = f"-c {c} -s 0 -v 5"
    train_acc=train(y_train.ravel(), X_train, params)
    Cross_Validation_accuracy.append(train_acc)
    print("Training Error=", 100-train_acc)
    print()


Training C=0.1
Cross Validation Accuracy = 95.8%
Training Error= 4.200000000000003

Training C=1
Cross Validation Accuracy = 96.6%
Training Error= 3.4000000000000057

Training C=10
Cross Validation Accuracy = 96.8%
Training Error= 3.200000000000003

Training C=100
Cross Validation Accuracy = 97%
Training Error= 3.0

Training C=1000
Cross Validation Accuracy = 97%
Training Error= 3.0



In [14]:
print("Best accuracy = ",max(Cross_Validation_accuracy), "%")
print("Best C value = ",C_list[Cross_Validation_accuracy.index(max(Cross_Validation_accuracy))])

Best accuracy =  97.0 %
Best C value =  100


In [15]:
error_rate=100-max(Cross_Validation_accuracy)
print("Error Rate Percentage for Breast Cancer Data(Training Data)=", error_rate, "%")

Error Rate Percentage for Breast Cancer Data(Training Data)= 3.0 %


In [16]:
model = train(y_train.ravel(), X_train, '-c 10 -s 0')
y_pred, _, _ = predict(y_test.ravel(), X_test, model)


Accuracy = 96.1749% (176/183) (classification)


In [17]:
acc = accuracy_score(y_test.tolist(), y_pred)
print(acc)

0.9617486338797814


In [18]:
error_rate=1-acc

In [19]:
error_rate_percent=error_rate*100
print("Error Rate Percentage for Breast Cancer Data=", error_rate_percent, "%")

Error Rate Percentage for Breast Cancer Data= 3.825136612021862 %


# **SONAR DATA:**

In [20]:
X_total, y_total=load_svmlight_file("/content/sonar_scale.txt")

In [21]:
X_total

<208x60 sparse matrix of type '<class 'numpy.float64'>'
	with 12479 stored elements in Compressed Sparse Row format>

In [22]:
train_indices = np.loadtxt('/content/sonar-scale-train-indices.txt', dtype=int)
test_indices = np.loadtxt('/content/sonar-scale-test-indices.txt', dtype=int)

In [23]:
X_train =X_total[train_indices - 1]
y_train =y_total[train_indices -1]
X_test  =X_total[test_indices - 1]
y_test  =y_total[test_indices - 1]

In [24]:
C_list = [0.1, 1, 10, 100, 1000]
Cross_Validation_accuracy=[]
for c in C_list:
    print(f"Training C={c}")
    params = f"-c {c} -s 0 -v 5"
    train_acc=train(y_train.ravel(), X_train, params)
    Cross_Validation_accuracy.append(train_acc)
    print("Training Error=", 100-train_acc)
    print()

Training C=0.1
Cross Validation Accuracy = 71.3333%
Training Error= 28.666666666666657

Training C=1
Cross Validation Accuracy = 74%
Training Error= 26.0

Training C=10
Cross Validation Accuracy = 74%
Training Error= 26.0

Training C=100
Cross Validation Accuracy = 71.3333%
Training Error= 28.666666666666657

Training C=1000
Cross Validation Accuracy = 70.6667%
Training Error= 29.33333333333333



In [25]:
print("Best accuracy = ",max(Cross_Validation_accuracy),"%")
print("Best C value = ",C_list[Cross_Validation_accuracy.index(max(Cross_Validation_accuracy))])

Best accuracy =  74.0 %
Best C value =  1


In [26]:
error_rate=100-max(Cross_Validation_accuracy)
print("Error Rate Percentage for Sonar Data(Training Data)=", error_rate, "%")

Error Rate Percentage for Sonar Data(Training Data)= 26.0 %


In [27]:
model = train(y_train.ravel(), X_train, '-c 10 -s 0')
y_pred, _, _ = predict(y_test.ravel(), X_test, model)

Accuracy = 72.4138% (42/58) (classification)


In [28]:
acc = accuracy_score(y_test.tolist(), y_pred)
print(acc)

0.7241379310344828


In [29]:
error_rate=1-acc
error_rate_percent=error_rate*100
print("Error Rate Percentage for Sonar Data=", error_rate_percent, "%")

Error Rate Percentage for Sonar Data= 27.586206896551722 %


# **ACCURACY COMPARISON**

\begin{array}{|c|c|c|c|} \hline
Data & Best C val & Training Error & Test Accuracy\\ \hline
Breast Cancer & 100 & 97.0\% & 96.17\%\\ \hline
Sonar & 1 & 74.0\% & 72.41\%\\ \hline
\end{array}

# **ERROR RATE COMPARISON (As per Question)**

**Calculated Error Rate=100 - Accuracy (in percentage)**

\begin{array}{|c|c|c|c|} \hline
Data & Best C val & Training Error & Test Error\\ \hline
Breast Cancer & 100 & 3.0\% & 3.83\%\\ \hline
Sonar & 1 & 26.0\% & 27.59\%\\ \hline
\end{array}

# ***OBSERVATION ANALYSIS***

The accuracy of a logistic regression model can vary based on the dataset because different datasets may have different characteristics, such as the number of features, the distribution of the features, and the class balance. These characteristics can affect the performance of the logistic regression model in several ways:

**Number of features**: The number of features in the dataset can affect the performance of the logistic regression model. If the dataset has a large number of features, the model may have a higher risk of overfitting, which can lead to poor performance on new, unseen data. On the other hand, if the dataset has a small number of features, the model may not be able to capture all the relevant information, which can also lead to poor performance.

**Distribution of the features**: The distribution of the features in the dataset can also affect the performance of the logistic regression model. If the features have different scales or distributions, some features may have a higher impact on the model than others. This can lead to a lower accuracy if important features are not properly weighted.

**Class balance**: The class balance in the dataset can also affect the performance of the logistic regression model. If the classes are imbalanced, with one class having significantly more samples than the other, the model may be biased towards the majority class. This can lead to a lower accuracy on the minority class.